<a href="https://colab.research.google.com/github/Jatchen/Acorn_Practicum/blob/main/0119_Acorn_Linear_Programming_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import ast  # Import the ast module for safe literal evaluation
import re  # Import the 're' module
import pulp # linear programming solver

In [ ]:
# read the data (whose format is different from orginal data Acorn provided )
result_2_df = pd.read_csv("/content/drive/MyDrive/result_2_df.csv")

In [ ]:
# Parameters
q = 40  # Maximum number of times a buyer is listed in the property recommendations

# Clear previous variables by creating a new problem object
prob = pulp.LpProblem("PropertyAssignmentProblem", pulp.LpMaximize)

In [ ]:
# Decision variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in result_2_df.property_id.unique() for j in result_2_df.customer_id.unique()), cat='Binary')

# Objective function
prob += pulp.lpSum([result_2_df.loc[(result_2_df.property_id == i) & (result_2_df.customer_id == j), 'rating'].values[0] * x[(i, j)]
                    if not result_2_df.loc[(result_2_df.property_id == i) & (result_2_df.customer_id == j), 'rating'].empty
                    else 0
                    for i in result_2_df.property_id.unique() for j in result_2_df.customer_id.unique()])

In [ ]:
# Constraints
for j in result_2_df.customer_id.unique():
    prob += pulp.lpSum([x[(i, j)] for i in result_2_df.property_id.unique()]) <= q

In [ ]:
# Solve the problem
prob.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[prob.status])

# Print the total utility
print("Total Utility of Assignments = ", pulp.value(prob.objective))

In [ ]:
# print the result

# Create a list to store the data
new_data = []

# Extract information from all variables
for v in prob.variables():
    buyer_id, property_id = map(int, re.findall(r'\d+', v.name))
    xij_value = v.varValue
    new_data.append((property_id, buyer_id, xij_value))

# Create a new DataFrame
new_df = pd.DataFrame(new_data, columns=['property_id', 'customer_id', 'Xij_value'])

# 交换 customer_id 和 property_id 列
new_df['customer_id'], new_df['property_id'] = new_df['property_id'], new_df['customer_id']

# merge rating to result
# 确保 'property_id' 和 'customer_id' 列在两个 DataFrame 中都是相同的数据类型
new_df['property_id'] = new_df['property_id'].astype('int64')
new_df['customer_id'] = new_df['customer_id'].astype('int64')
result_2_df['property_id'] = result_2_df['property_id'].astype('int64')
result_2_df['customer_id'] = result_2_df['customer_id'].astype('int64')

merged_df = pd.merge(new_df, original_2_df, on=['property_id', 'customer_id'], how='inner')
print(merged_df)

In [ ]:
# to test if meet  property limitation

# filter succseful assign out
filtered_df = merged_df[merged_df['Xij_value'] == 1]

# Count each customer_ Different properties corresponding to IDs_ Number of IDs
result = filtered_df.groupby('customer_id')['property_id'].nunique().reset_index()

# rpint the result
print(result)